In [1]:
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import os


<h3> Setting up data 

In [34]:
print(os.listdir("../MLDS"))

#import and shuffle training data
original_data = pd.read_csv("../MLDS/training_dataset.csv").sample(frac=1).reset_index(drop=True)


#get rid of text columns 
data = original_data.drop(columns=["text"])


#set prose to 0 and poetry to 1 
for x in range(0,469):
    if(data.iloc[x,27]=='prose'):
        data.iloc[x,27] = 0
    else:
        data.iloc[x,27] = 1


print(original_data.iloc[:,0].std())

['MLDS_Competition.ipynb', 'training_dataset.csv', 'General_LR.ipynb', 'Decision Boundaries.ipynb', '.DS_Store', 'savedModel1.pt', 'try1.pt', 'test_dataset.csv', '.ipynb_checkpoints', 'FeatureDescriptions.csv', 'sample_submission.csv']
135.53289883517826


In [3]:
#data

In [3]:
X_train = torch.tensor(data.iloc[0:350,1:27].values).float()
X_val = torch.tensor(data.iloc[350:469,1:27].values)


Y_train = torch.tensor(data.iloc[0:350,27].values)
Y_train = Y_train.view(350,-1)
Y_val = torch.tensor(data.iloc[351:469,27].values)
Y_val = Y_val.view(118,-1) #PROBLEM!!!!!!!

print(X_train.size())
print(X_val.size())


torch.Size([350, 26])
torch.Size([119, 26])


<h3> Creating Models, Optimzers, Trainers, Cost Functions, etc


In [4]:
class simple_logreg(nn.Module):
    
    def __init__(self,inputs):
        super(simple_logreg, self).__init__()
        self.linear = nn.Linear(inputs,1)
        
    def forward(self,x):
        return torch.sigmoid(self.linear(x))

    
model = simple_logreg(20)

optimizer = torch.optim.SGD(model.parameters(), lr=0.5, momentum = 0.5)

cost = nn.BCELoss()




def batch_GD(epochs):
    for e in range (epochs):
        optimizer.zero_grad()
        prediction = model(X_train.float()[:,0:20])
        loss = cost(prediction,Y_train.float())
        loss.backward()
        optimizer.step()
        
        #debugging
        #print("Prediction: ", prediction[0:5])
        #print("Answer: ", Y_train.float()[0:5])
        #print("Grads of: " , model.state_dict())
        #for p in model.parameters():
            #print(p.grad)
        
        print("Loss: ",loss)
    
def stochastic_GD(epochs):
    for epoch in range(epochs):
        totalCost = 0
        for i in range(350):
            optimizer.zero_grad()
            pred = model(X_train[i,0:20].float())
            loss = cost(pred,Y_train[i].float())
            totalCost+=loss
            loss.backward()
            optimizer.step()
            

        if(epoch%500==0):
            print("Cost: ", totalCost)
            print(model.state_dict())
        


In [48]:
def noprintstochastic_GD(epochs):
    for epoch in range(epochs):
        totalCost = 0
        for i in range(350):
            optimizer.zero_grad()
            pred = model(X_train[i,0:20].float())
            loss = cost(pred,Y_train[i].float())
            totalCost+=loss
            loss.backward()
            optimizer.step()
            

optimizer = torch.optim.SGD(model.parameters(), lr=1, momentum = 0.5)
noprintstochastic_GD(10000)

In [60]:
torch.save(model.state_dict(), "../MLDS/try1.pt")

In [73]:
print(cost(model(X_val[1:,0:20].float()),Y_val.float()))

tensor(0.1186, grad_fn=<BinaryCrossEntropyBackward>)


/anaconda2/envs/learningML/lib/python3.5/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([118])) that is different to the input size (torch.Size([118, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


In [59]:
print(Y_val[0:30])

tensor([1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
        0, 1, 1, 1, 0, 1])


In [15]:
savedModel = simple_logreg(20)
savedModel.load_state_dict(torch.load('savedModel1.pt'))

In [23]:
numRight = 0
total = 0
newPred = savedModel(X_val[1:,0:20].float())
for x in range(118):
    if(newPred[x]>0.5):
        newPred[x] = 1
    else:
        newPred[x] = 0
#print(newPred)

for x in range(118):
    if (newPred[x].float()==Y_val[x].float()):
        numRight = numRight + 1
    total = total + 1

print("Accuracy: ",  numRight/total)

Accuracy:  0.9745762711864406


In [22]:
#95% accuracy yay 

In [24]:
def savedModelGD(epochs):
    for epoch in range(epochs):
        totalCost = 0
        for i in range(350):
            optimizer.zero_grad()
            pred = savedModel(X_train[i,0:20].float())
            loss = cost(pred,Y_train[i].float())
            totalCost+=loss
            loss.backward()
            optimizer.step()
        
        if(epoch%500==0):
            print("Cost: ", totalCost)

optimizer = torch.optim.SGD(savedModel.parameters(), lr=1, momentum = 0.5)
savedModelGD(1000)

Cost:  tensor(28.9740, grad_fn=<AddBackward0>)
Cost:  tensor(29.7459, grad_fn=<AddBackward0>)


In [14]:
torch.save(savedModel.state_dict(), "../MLDS/savedModel1.pt")

In [15]:
#todo: Keep running the above method and taking cost of saved Model down. Parameters are working well but we haven't reached min yet